I'm biginner of kaggle competition.

I made this basic Notebook and running it. 
After I ran my model, I could make my submission.csv which written 0 or 1. 
After I submitted my submission file, however the public score wasn't calculated. 
This Error name is "Notebook Timeout error". 
Because of my few nowledge, now I can't solve this Notebook Timeout error.

I want to know what is wrong and how to fix it.


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import janestreet
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Input
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss, accuracy_score

#input data
path = "../input/jane-street-market-prediction/"
train = pd.read_csv(path + "train.csv",nrows=10000)
train = train.astype({c: np.float32 for c in train.select_dtypes(include='float64').columns})

column = ['feature_0', 'feature_1', 'feature_2',
       'feature_3', 'feature_4', 'feature_5', 'feature_6', 'feature_7',
       'feature_8', 'feature_9', 'feature_10', 'feature_11', 'feature_12',
       'feature_13', 'feature_14', 'feature_15', 'feature_16',
       'feature_17', 'feature_18', 'feature_19', 'feature_20',
       'feature_21', 'feature_22', 'feature_23', 'feature_24',
       'feature_25', 'feature_26', 'feature_27', 'feature_28',
       'feature_29', 'feature_30', 'feature_31', 'feature_32',
       'feature_33', 'feature_34', 'feature_35', 'feature_36',
       'feature_37', 'feature_38', 'feature_39', 'feature_40',
       'feature_41', 'feature_42', 'feature_43', 'feature_44',
       'feature_45', 'feature_46', 'feature_47', 'feature_48',
       'feature_49', 'feature_50', 'feature_51', 'feature_52',
       'feature_53', 'feature_54', 'feature_55', 'feature_56',
       'feature_57', 'feature_58', 'feature_59', 'feature_60',
       'feature_61', 'feature_62', 'feature_63', 'feature_64',
       'feature_65', 'feature_66', 'feature_67', 'feature_68',
       'feature_69', 'feature_70', 'feature_71', 'feature_72',
       'feature_73', 'feature_74', 'feature_75', 'feature_76',
       'feature_77', 'feature_78', 'feature_79', 'feature_80',
       'feature_81', 'feature_82', 'feature_83', 'feature_84',
       'feature_85', 'feature_86', 'feature_87', 'feature_88',
       'feature_89', 'feature_90', 'feature_91', 'feature_92',
       'feature_93', 'feature_94', 'feature_95', 'feature_96',
       'feature_97', 'feature_98', 'feature_99', 'feature_100',
       'feature_101', 'feature_102', 'feature_103', 'feature_104',
       'feature_105', 'feature_106', 'feature_107', 'feature_108',
       'feature_109', 'feature_110', 'feature_111', 'feature_112',
       'feature_113', 'feature_114', 'feature_115', 'feature_116',
       'feature_117', 'feature_118', 'feature_119', 'feature_120',
       'feature_121', 'feature_122', 'feature_123', 'feature_124',
       'feature_125', 'feature_126', 'feature_127', 'feature_128',
       'feature_129']

#train data
train['action'] =  (  (train['resp_1'] > 0 ) & (train['resp_2'] > 0 ) & (train['resp_3'] > 0 ) & (train['resp_4'] > 0 ) &  (train['resp'] > 0 )   ).astype('int')
train.fillna(0,inplace=True)
train.drop(columns = ["resp_1","resp_2","resp_3","resp_4","resp",'date','weight','ts_id'],inplace=True)
train_y = train[["action"]].copy()
train_x = train.copy()
train_x.drop("action",axis = 1,inplace = True)

#学習させる
def create_model(df):
    model = Sequential()
    model.add(BatchNormalization())
    model.add(Dense(1024, activation='relu',input_shape=(df.shape[1],)))
    model.add(Dropout(0.5))
    model.add(BatchNormalization())    
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer="adam",
                  loss='binary_crossentropy', 
                  metrics=['accuracy'])
    return model

score_logloss = []
batch_size = 1024
epochs = 250

#learning
score,score_logloss = [],[]
kf = KFold(n_splits=4,shuffle = True,random_state=71)
for tr_idx,va_idx in kf.split(train_x):
    tr_x,va_x = train_x.iloc[tr_idx],train_x.iloc[va_idx]
    tr_y,va_y = train_y.iloc[tr_idx],train_y.iloc[va_idx]
    model = create_model(train_x)
        
    history = model.fit(tr_x.values,tr_y.values,batch_size = batch_size,epochs = epochs,
                        verbose = 1,validation_data=(va_x,va_y))
    
    va_pred = model.predict(va_x)
    score = log_loss(va_y,va_pred)
    score_logloss.append(score)

    
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set
for (test_df, sample_prediction_df) in iter_test:
    test_df.fillna(0,inplace = True)
    X_test = test_df.loc[:, column].values
    y_preds = model.predict(X_test)
    sample_prediction_df.action = np.where(y_preds >= 0.5, 1, 0).astype(int)
    env.predict(sample_prediction_df)

print("X_test",X_test.shape,X_test)




